In this example, S3-CIMA is from plotting_sb import plot_results
to the data of the CRC study (Schürch et al., 2020), where patients were stratified in two groups with differential survival, i.e., 17 patients with Crohn’s-like reaction (CLR) and 18 patients with diffuse inflammatory infiltration (DII)including 56-marker multiplexed CODEX tissue imaging data of140 TMA spots i.e., 4 CODEX images for each patient

In [7]:
import os
import numpy as np
import pandas as pd
from grepfunc import grep
import argparse
import modules_scima as ms
from scipy import stats
import pickle
from utils import mkdir_p
from plotting_scima import plot_results

np.random.seed(12345)

Read the data table including ALL cells ids, marker expression, image spot ids, patient ids, X and Y coordinates.

In [7]:
path = '/'.join(os.path.abspath(os.getcwd()).split('/')[:-1])+'/CRCs'

#remove dirt
Data = pd.read_csv("CrCnodirt.csv", sep=',')


In [8]:
data =Data.values

cols =Data.columns.tolist()

inx = np.asarray(grep(cols, "Cyc" , n=True))
markers = inx[:,1].tolist()
inx = inx [:,0].astype(int)

Intensity =data[:,inx]
#Intensity = np.arcsinh(Intensity.astype(float))

Intensity = (1e-3+Intensity).astype(float)
Intensity = np.log2(Intensity)
Intensity =stats.zscore(pd.DataFrame(Intensity))

In [9]:

pat = np.asarray(Data['patients'])
image = np.asarray(Data['spots'])
ct = np.asarray(Data['ClusterName'])
x = np.asarray(Data['X.X'])
y = np.asarray(Data['Y.Y'])
cellid = np.asarray(Data['CellID'])
groups = np.asarray(Data['groups'])
labels = np.unique(np.asarray(Data['groups']))


Make the spatial input for global spatial enrichment analysis (back ground (BG))

In [10]:
#number of random sets
N = 50
#number of nearest neighbor cells 
K = 50
Anchor = 'BG'
OUTDIR = path + '/Anchor' + str(Anchor)

for i in range(0, len(labels)):
    inx = np.where(groups == labels[i])[0]
    rd,rs, nset = ms.BG_spatial_input_per_sample(N, image[inx], pat[inx], Intensity[inx,:], ct[inx], x[inx], 
                               y[inx], cellid[inx], K, labels[i], OUTDIR)


In [2]:

Anchor = 'BG'
ntrain_per_class = 12
K = 50 
k = 50


labels = ['1', '2']
classes = [0, 1]

path = '/'.join(os.path.abspath(os.getcwd()).split('/')[:-1])+'/CRCs'

nrun = 50
nset_thr = 1
background = False
ms.run_scima(Anchor, ntrain_per_class, K, k, nset_thr, labels, classes, path, nrun, background)



output will be rdirected to /Users/sepidehbabaei/Desktop/CRCs/resultsBG/run_log//data_run_log_18_18_27.txt


Make the spatial input for local spatial enrichment analysis (e.g. CF4 T cells as an anchor)

In [15]:
CTs = np.unique(np.asarray(Data['ClusterName']))
Anchor = 'CD4+ T cells'
OUTDIR = path + '/Anchor' + str(Anchor)

#number of nearest neighbor cells around each anchor cell
K = 50

for i in range(0, len(labels)):
    inx = np.where(groups == labels[i])[0]
    d,s,rd,rs, nset = ms.spatial_input_per_sample(Anchor,image[inx], pat[inx], Intensity[inx,:], ct[inx], 
                                                  x[inx], y[inx], cellid[inx], K, labels[i], OUTDIR)

In [9]:

Anchor = 'CD4+ T cells'
ntrain_per_class = 12
K = 50 
k = 10

# if background sets used the order should be first the sample labels and the random set at the end. Forexample here,
# the background label is 2.
labels = ['1', '2', 'rand1', 'rand2']
classes = [0, 1, 2, 2]

path = '/'.join(os.path.abspath(os.getcwd()).split('/')[:-1])+'/CRCs'

nrun = 50
nset_thr = 0.9
background = True
ms.run_scima(Anchor, ntrain_per_class, K, k, nset_thr, labels, classes, path, nrun, background)



output will be rdirected to /Users/sepidehbabaei/Desktop/CRCs/resultsCD4+ T cells/run_log//data_run_log_18_46_19.txt


In [8]:
OUTDIR = '/Users/sepidehbabaei/Desktop/CRCs/resultsCD4+ T cells/out_18_30_20'
print(OUTDIR)

results=pickle.load(open(OUTDIR+'/model/results.p', 'rb'))                   # model result file

Data = pd.read_csv("CrCnodirt.csv", sep=',')
data =Data.values
cols =Data.columns.tolist()
inx = np.asarray(grep(cols, "Cyc" , n=True))
markers = inx[:,1].tolist()
inx = inx [:,0].astype(int)

Intensity =data[:,inx]
Intensity = (1e-3+Intensity).astype(float)
Intensity = np.log2(Intensity)
Intensity =stats.zscore(pd.DataFrame(Intensity))
samples = Intensity

phenotypes = np.asarray(Data['groups'])
pat = np.asarray(Data['patients'])

analysis_outdir=os.path.join(OUTDIR,'analysis/')
mkdir_p(analysis_outdir)

print('plotting the test results ....')
_ = plot_results(results, samples, phenotypes, markers, analysis_outdir, 
                 pat, filter_diff_thres=0.2, filter_response_thres=0.0, ALL = True)
                 

/Users/sepidehbabaei/Desktop/CRCs/resultsCD4+ T cells/out_18_30_20
plotting the test results ....
number of measured markers 56
keep_idx [1]
sample size 250864
patient id list Counter({18: 11564, 17: 10483, 4: 10070, 9: 9983, 30: 9383, 5: 9149, 15: 9051, 21: 9040, 16: 8996, 14: 8837, 13: 8378, 8: 8372, 23: 8086, 3: 7671, 31: 7559, 26: 7502, 12: 6762, 7: 6664, 10: 6466, 25: 6356, 19: 6226, 24: 6184, 20: 6173, 28: 6140, 6: 5900, 2: 5862, 32: 5822, 11: 5810, 33: 5542, 1: 5452, 35: 5130, 29: 4921, 27: 4049, 22: 3908, 34: 3373})
thresh: 1.7714963223153641
